### Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
import os
import PIL
import PIL.Image
import tensorflow as tf

print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPU visible:", tf.config.list_physical_devices('GPU'))

2025-11-12 22:49:51.724538: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-12 22:49:51.761511: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-12 22:49:52.448431: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Built with CUDA: True
GPU visible: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import numpy as np
import pandas as pd
import os
import PIL
import PIL.Image
import tensorflow as tf

ROOT = "./face_skin_dataset/"
BATCH_SIZE = 32
IMG_SIZE = (512, 512)
AUTOTUNE = tf.data.AUTOTUNE # automatically tunes CPU usage

train_annotations = pd.read_csv(os.path.join(ROOT, "train", "_annotations.csv"))
val_annotations = pd.read_csv(os.path.join(ROOT, "valid", "_annotations.csv"))
test_annotations = pd.read_csv(os.path.join(ROOT, "test",  "_annotations.csv"))

# create path/to/img lists
def make_paths(df, split):
    return [os.path.join(ROOT, split, fname) for fname in df["filename"]]

train_paths = make_paths(train_annotations, "train")
val_paths = make_paths(val_annotations,   "valid")
test_paths = make_paths(test_annotations,  "test")

# creating labels lists
class_names = sorted(train_annotations["class"].unique())
num_classes = len(class_names)
name2id = {name: i for i, name in enumerate(class_names)}

train_labels = train_annotations["class"].map(name2id).astype("int32")
val_labels   = val_annotations["class"].map(name2id).astype("int32")
test_labels  = test_annotations["class"].map(name2id).astype("int32")

# create a tensorFlow dataset with ( path/to/img, label) pairs
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))

I0000 00:00:1763016593.356753    5548 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6071 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5


In [3]:
def load_image(path, label):
    image = tf.io.read_file(path) # read image from disk
    image = tf.image.decode_jpeg(image, channels=3) # decode jpeg image
    image = tf.image.resize(image, IMG_SIZE) # ensure images 512 x 512
    image = tf.image.convert_image_dtype(image, tf.float32) # normalize image pixel scale from [0, 255] to [0, 1]
    return image, label

# apply load_image function to each image in dataset using .map function
# shuffle images and batch in sets of 32
def pipeline(paths, labels, training):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if training:
        ds = ds.shuffle(buffer_size=min(len(paths), 1000), reshuffle_each_iteration=True)
    ds = ds.map(load_image, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(AUTOTUNE)
    return ds

train_ds = pipeline(train_paths, train_labels, True)
val_ds = pipeline(val_paths, val_labels, False)
test_ds = pipeline(test_paths, test_labels, False)

### Model Architecture

In [4]:
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

model = models.Sequential(
    [
        # convolution 1
        layers.Conv2D(filters=64, kernel_size=7, strides=2, padding="same", input_shape=(512,512,3)),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # pool 1
        layers.MaxPooling2D(pool_size=3, strides=2),

        # convolution 2
        layers.Conv2D(filters=128, kernel_size=3, padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        
        # convolution 3
        layers.Conv2D(filters=128, kernel_size=3, padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        
        # pool 2
        layers.MaxPooling2D(pool_size=3, strides=2),
        
        # convolution 3     
        layers.Conv2D(filters=256, kernel_size=3, padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # convolution 4
        layers.Conv2D(filters=256, kernel_size=3, padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # pool 3        
        layers.MaxPooling2D(pool_size=3, strides=2),

        # fully connected layers
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ]
)

/home/wtl04/coding/uni/cecs458/inclass/regularization/tf_venv/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 127, 127, 128)  │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 127, 127, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 127, 127, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 127, 127, 128)  │       147,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 127, 127, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 127, 127, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 63, 63, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 63, 63, 256)    │       294,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 63, 63, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 63, 63, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 63, 63, 256)    │       589,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 63, 63, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 63, 63, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 19)             │         2,45

 Total params: 1,154,067 (4.40 MB)

 Trainable params: 1,152,403 (4.40 MB)

 Non-trainable params: 1,664 (6.50 KB)

In [6]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

### Model Training

In [7]:
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10


2025-11-12 22:49:55.959024: I external/local_xla/xla/service/service.cc:163] XLA service 0x710f10015020 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-12 22:49:55.959046: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 SUPER, Compute Capability 7.5
2025-11-12 22:49:56.006317: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-12 22:49:56.284852: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
2025-11-12 22:49:57.648677: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.05GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-11-12 22:49:58.267149: W 

  1/447 ━━━━━━━━━━━━━━━━━━━━ 2:11:27 18s/step - accuracy: 0.0000e+00 - loss: 3.4737

I0000 00:00:1763016611.615098    5610 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


447/447 ━━━━━━━━━━━━━━━━━━━━ 112s 212ms/step - accuracy: 0.5518 - loss: 1.3754 - val_accuracy: 0.3895 - val_loss: 4.3051
Epoch 2/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 85s 189ms/step - accuracy: 0.5857 - loss: 1.2195 - val_accuracy: 0.3009 - val_loss: 1.8718
Epoch 3/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 83s 187ms/step - accuracy: 0.6013 - loss: 1.1554 - val_accuracy: 0.3358 - val_loss: 7.6299
Epoch 4/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 85s 190ms/step - accuracy: 0.6071 - loss: 1.1308 - val_accuracy: 0.4549 - val_loss: 1.5064
Epoch 5/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 84s 189ms/step - accuracy: 0.6174 - loss: 1.0802 - val_accuracy: 0.4608 - val_loss: 1.6933
Epoch 6/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 83s 186ms/step - accuracy: 0.6309 - loss: 1.0526 - val_accuracy: 0.4535 - val_loss: 1.4915
Epoch 7/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 80s 179ms/step - accuracy: 0.6344 - loss: 1.0341 - val_accuracy: 0.5305 - val_loss: 1.3665
Epoch 8/10
447/447 ━━━━━━━━━━━━━━━━━━━━ 83s 187ms/step - accuracy: 0.6344 - loss: 1.0247 - va

### Model Evaluation

In [8]:
model.evaluate(test_ds)

25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 218ms/step - accuracy: 0.5195 - loss: 1.8286


[1.8286230564117432, 0.5194968581199646]